In [92]:
import findspark
findspark.init()
from pyspark import SparkContext

In [93]:
sc = SparkContext(appName='part-2')

In [94]:
csv = sc.textFile('AllVideos_short.csv')

In [95]:
output_path = 'output'

In [96]:
csv.take(2)

['video_id,trending_date,category_id,category,publish_time,views,likes,dislikes,comment_count,ratings_disabled,video_error_or_removed,country',
 'SbOwzAl9ZfQ,17.14.11,24,Entertainment,2017-11-13T06:06:22.000Z,310130,4182,361,1836,False,False,MX']

In [97]:
def mapper(line):
    line = line.strip()
    part = line.split(',')
    vid = part[0].strip()
    trending_date = part[1].strip()
    category = part[3].strip()
    likes = part[6].strip()
    dislikes = part[7].strip()
    country = part[11].strip()
    key = vid + '/' + country + '/' + category
    value = trending_date + '/' + likes + '/' + dislikes
    return key, value

In [98]:
csvNoHead = csv.zipWithIndex().filter(lambda tup: tup[1] > 0).keys()

after_map = csvNoHead.map(mapper)

In [99]:
after_map.collect()[0:10]

[('SbOwzAl9ZfQ/MX/Entertainment', '17.14.11/4182/361'),
 ('klOV6Xh-DnI/MX/People & Blogs', '17.14.11/271/174'),
 ('6L2ZF7Qzsbk/MX/News & Politics', '17.14.11/10105/266'),
 ('hcY52MFWMDM/MX/News & Politics', '17.14.11/378/171'),
 ('_OXDcGPVAa4/MX/Howto & Style', '17.14.11/57781/681'),
 ('Q9kK6NWZR1U/MX/Music', '17.14.11/506/67'),
 ('c9VTD3n_IDs/MX/People & Blogs', '17.14.11/2277/69'),
 ('XzULSsZYMRc/MX/News & Politics', '17.14.11/7745/659'),
 ('uijjYNtl_UM/MX/Entertainment', '17.14.11/20155/912'),
 ('cOJ68MQm2ac/MX/Entertainment', '17.14.11/83582/2194')]

In [100]:
def calculate_difference(value):
    line_in_iterator = [i for i in value]
    likes_dislikes = [[0,0],[0,0]]
    difference = [0,0]
    if len(line_in_iterator) >1:
        for i in range(2):
            split = line_in_iterator[i].split("/")
            likes_dislikes[i][0] = int(split[1])
            likes_dislikes[i][1] = int(split[2])
        
        difference[0] = likes_dislikes[1][0] - likes_dislikes[0][0]
        difference[1] = likes_dislikes[1][1] - likes_dislikes[0][1]
        result = difference[1] - difference[0]
        return result
    return 0

        

In [101]:
after_calculate = after_map.groupByKey().mapValues(calculate_difference)


In [102]:
after_calculate.collect()[:10]

[('SbOwzAl9ZfQ/MX/Entertainment', -1517),
 ('klOV6Xh-DnI/MX/People & Blogs', 0),
 ('_OXDcGPVAa4/MX/Howto & Style', -34377),
 ('Q9kK6NWZR1U/MX/Music', 0),
 ('c9VTD3n_IDs/MX/People & Blogs', -1463),
 ('XzULSsZYMRc/MX/News & Politics', 0),
 ('uijjYNtl_UM/MX/Entertainment', 0),
 ('rZZEeeAVgog/MX/Comedy', 0),
 ('kTT472QeJGg/MX/People & Blogs', 0),
 ('7jmJtdqI6YE/MX/Entertainment', -116)]

In [103]:
sortedByResult = after_calculate.sortBy(lambda a: a[1],0)

In [104]:
sortedByResult.collect()[:10]

[('BEePFpC9qG8/DE/Film & Animation', 366556),
 ('RmZ3DPJQo2k/KR/Music', 334594),
 ('1Aoc-cd9eYs/GB/Entertainment', 192222),
 ('QwZT7T-TXT0/US/Entertainment', 189608),
 ('QwZT7T-TXT0/GB/Entertainment', 189605),
 ('PfLCyR6Efvw/GB/Music', 106418),
 ('ZGEoqPpJQLE/DE/Music', 98934),
 ('ZGEoqPpJQLE/RU/Music', 98930),
 ('84LBjXaeKk4/RU/Entertainment', 93961),
 ('84LBjXaeKk4/FR/Entertainment', 93961)]

In [105]:
def mapToFormat(line):
    overall = line[0]
    result = line[1]
    splited = overall.split('/')
    vid = splited[0]
    country = splited[1]
    category = splited[2]
    key = vid
    value = str(result) + ',' + category + ',' + country
    return key, value
    

In [106]:
answer = sortedByResult.map(mapToFormat)

In [118]:
output = answer.collect()[:10]

In [ ]:
output1 = sc.parallelize(output)

In [120]:
output1.saveAsTextFile(output_path)

In [91]:
sc.stop()